In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("Train.csv")


Checking for null values

In [ ]:
data.isnull().sum()

I have chosen the threshold of missing values in a row to be 0.7 (meaning if a row has more than 30% empty values it will be dropped)

Thus....

Dropping all the rows with missing value rate higher than threshold


In [ ]:
data = data.loc[data.isnull().mean(axis=1) < 0.7]

In [ ]:
data.isnull().sum()

Picking the features i would use to make the model

In [ ]:
data = data[['TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'REGULARITY','TOP_PACK','FREQ_TOP_PACK', 'CHURN']]

Filling the remaining empty rows with the mean in every respective column

In [ ]:
data = data.fillna(data[['MONTANT','FREQUENCE_RECH','REVENUE','ARPU_SEGMENT','FREQUENCE','DATA_VOLUME','ON_NET','ORANGE','TIGO','FREQ_TOP_PACK']].mean())

In [ ]:
data.isnull().sum()

Fill TOP_PACK column with the most frequent value

In [ ]:
data = data.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [ ]:
data.isnull().sum()

Pick train features and target feature (which is obviously CHURN)

In [ ]:
trainx = data[['TENURE','MONTANT','FREQUENCE_RECH','REVENUE','ARPU_SEGMENT','FREQUENCE','REGULARITY']]
trainy = data['CHURN']

CatBoost is a machine learning algorithm that uses gradient boosting on decision trees. It is available as an open source library.

https://catboost.ai/en/docs/https://catboost.ai/en/docs/


In [ ]:
from catboost import CatBoostClassifier

cat = [0]

model = CatBoostClassifier(iterations=10)



In [ ]:
model.fit(trainx, trainy, cat)

In [ ]:
testdata = pd.read_csv("Test.csv")

In [ ]:
testdata2 = testdata[['TENURE','MONTANT','FREQUENCE_RECH','REVENUE','ARPU_SEGMENT','FREQUENCE','REGULARITY']]

In [ ]:
testdata2 = testdata2.fillna(testdata2[['MONTANT','FREQUENCE_RECH','REVENUE','ARPU_SEGMENT','FREQUENCE']].mean())

In [ ]:
testdata2 = testdata2.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [ ]:
churn = model.predict_proba(testdata2)[:,1]

In [ ]:
churn.tolist()

In [ ]:
testdata.insert(2, "CHURN", churn, True)

In [ ]:
testdata.drop(['REGION','TENURE','MONTANT','FREQUENCE_RECH','REVENUE','ARPU_SEGMENT','FREQUENCE','DATA_VOLUME','ON_NET','ORANGE','TIGO','ZONE1','ZONE2','MRG','REGULARITY','TOP_PACK','FREQ_TOP_PACK'], axis=1, inplace=True)

In [ ]:
testdata.head()

In [ ]:
testdata.to_csv('predictions.csv' ,index=False)